## Credit Card Fraud Dectection

This will use machine learning and predictive modeling to help detect fradulent credit card transaction.
The original data csv file can be found here: https://www.kaggle.com/mlg-ulb/creditcardfraud
The columns with the first five rows can be seen below in the next section.

In [11]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from statistics import mean
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.datasets import make_classification
from sklearn import ensemble
import warnings
warnings.filterwarnings('ignore')

kaggleFile=r'/kaggle/input/creditcardfraud/creditcard.csv'
data=pd.read_csv(kaggleFile)
print(data.head())

#### Find missing data

In [12]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = data.select_dtypes(include=numerics)

print("Missing data by column:")
findNA=df.isnull().sum().sort_values(ascending=False)/len(data)
print(findNA.head()) #There are no missing values

#### Correlations using heatmap
No multicollinearity was detected. 

In [22]:
def printHeat():
    corr = df.corr()
    #print(corr)
    highly_corr_features = corr.index[abs(corr["Class"])>0.2]
    plt.figure(figsize=(10,10))
    heat = sns.heatmap(data[highly_corr_features].corr(),annot=True,cmap="RdYlGn")
    top10=corr["Class"].sort_values(ascending=False).head(10)
    print(heat)
    #print(top10) # top ten correlations

#print correlation between features
def printHighCorr(df, features, threshold=0.7):
    print("Highly correlated variables above: ", threshold)
    corr_df = df[features].corr() # get correlations
    correlated_features = np.where(np.abs(corr_df) > threshold) # select ones above the abs threshold
    correlated_features = [(corr_df.iloc[x,y], x, y) for x, y in zip(*correlated_features) if x != y and x < y] # avoid duplication
    s_corr_list = sorted(correlated_features, key=lambda x: -abs(x[0])) # sort by correlation value
    correlation_df={}
    if s_corr_list == []:
        print("There are no highly correlated features with correlation above", threshold)
    else:
        for v, i, j in s_corr_list:
            correlation_df[corr_df.index[i] +" and "+ corr_df.columns[j]]= v
        correlation_df=pd.DataFrame(correlation_df,index=['Correlation'])
    return  correlation_df.T.sort_values(by='Correlation',ascending=False)

#Turn functions on or off below:
printHeat() 
#printHighCorr(data,data.columns).style.set_properties(**{'background-color': 'black','color': 'white'})

#### Split Data
Use 70% of the train data to predict the accuracy of the remaining 30% of the test data.

In [14]:
X=df.drop('Class', axis=1)
y=df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

#### Logistic Regression Predictive Model

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logReg = LogisticRegression()

#Train the model and create predictions
logReg.fit(X_train, y_train)
predictions = logReg.predict(X_test)

#use model to predict probability that given y value is 1:
y_pred_proba = logReg.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = round( metrics.roc_auc_score(y_test, y_pred_proba), 4 ) 
print("AUC is: ", auc)

#### Resources
1. https://www.kaggle.com/jdelamorena/recall-97-by-using-undersampling-neural-network